In [1]:
# Import another usefull things
from IPython.display import display
from IPython.display import Markdown
# Import the modules
import requests
import json

RW [prior work](https://docs.google.com/document/d/1Z1-vWOPv_iqa6GxJ7KKiujJ4O-jKnNu2avtXY_q8rx8/edit#)
## Querying API

##### The API is not supporting subqueries and some postgis functions in this moment. We would try to do some queries and if any is not implemented, we will do it directly to Carto

In [2]:
#Simple query to the API
def queryRaster(datasetId: str, query: str, geostore: str=None)-> dict:
    try:
        # fc8cf356-f4be-4635-a896-fb468aaaa832?sql=SELECT+ST_METADATA(rast)+FROM+users/resourcewatch/for_013_forest_landscape_restoration_opportunity
        URL = f'https://api.resourcewatch.org/v1/query/{datasetId}'
        params ={
            'sql':f'{query}'
        }
        if geostore:
            params['geostore']=geostore
            
        r = requests.get(URL, params = params)
        r.raise_for_status()
        return r.json()
    except requests.exceptions.HTTPError as err:
        print(err)

## Raster 
### Gee image and Carto rasters Remember no joins or subqueries allowed; 

In [52]:
display(Markdown('#### Function ST_METADATA'))
queryRaster('fc8cf356-f4be-4635-a896-fb468aaaa832', "SELECT ST_METADATA(rast) FROM 'users/resourcewatch/for_013_forest_landscape_restoration_opportunity'")

#### Function ST_METADATA

{'data': [{'st_metadata': {'bands': [{'dataType': {'precision': 'INT',
       'range': {'max': 255}},
      'grid': {'affineTransform': {'scaleX': 0.00833333333333333,
        'scaleY': -0.00833333333333333,
        'translateX': -180.00000000000003,
        'translateY': 90.00000000000001},
       'crsCode': 'EPSG:4326',
       'dimensions': {'height': 21600, 'width': 43200}},
      'id': 'b1',
      'pyramidingPolicy': 'MEAN'}],
    'geometry': {'coordinates': [[['-Infinity', '-Infinity'],
       ['Infinity', '-Infinity'],
       ['Infinity', 'Infinity'],
       ['-Infinity', 'Infinity'],
       ['-Infinity', '-Infinity']]],
     'type': 'Polygon'},
    'id': 'users/resourcewatch/for_013_forest_landscape_restoration_opportunity',
    'name': 'projects/earthengine-legacy/assets/users/resourcewatch/for_013_forest_landscape_restoration_opportunity',
    'sizeBytes': '19815070',
    'type': 'IMAGE',
    'updateTime': '2017-10-04T17:39:00.405273Z'}}],
 'meta': {}}

In [53]:
display(Markdown('#### Function ST_BANDMETADATA'))
queryRaster('fc8cf356-f4be-4635-a896-fb468aaaa832', "SELECT ST_BANDMETADATA(rast, b1) FROM 'users/resourcewatch/for_013_forest_landscape_restoration_opportunity'")

#### Function ST_BANDMETADATA

{'data': [{'st_bandmetadata': {'dataType': {'precision': 'INT',
     'range': {'max': 255}},
    'grid': {'affineTransform': {'scaleX': 0.00833333333333333,
      'scaleY': -0.00833333333333333,
      'translateX': -180.00000000000003,
      'translateY': 90.00000000000001},
     'crsCode': 'EPSG:4326',
     'dimensions': {'height': 21600, 'width': 43200}},
    'id': 'b1',
    'pyramidingPolicy': 'MEAN'}}],
 'meta': {}}

In [54]:
display(Markdown('#### Function st_summarystats'))
queryRaster('fc8cf356-f4be-4635-a896-fb468aaaa832', "SELECT st_summarystats(rast, 1, true) FROM 'users/resourcewatch/for_013_forest_landscape_restoration_opportunity'", '605fcf0e44c02de39fcaa7e30884be07')

#### Function st_summarystats

{'data': [{'st_summarystats': {'b1': {'count': 100044,
     'max': 4,
     'mean': 1.3628535231081242,
     'min': 0,
     'stdev': 1.0964333119818819,
     'sum': 135432.10980392157}}}],
 'meta': {}}

In [55]:
display(Markdown('#### Function ST_HISTOGRAM'))
queryRaster('fc8cf356-f4be-4635-a896-fb468aaaa832', "SELECT ST_HISTOGRAM(rast, 1, auto, true) FROM 'users/resourcewatch/for_013_forest_landscape_restoration_opportunity'", '605fcf0e44c02de39fcaa7e30884be07')

#### Function ST_HISTOGRAM

{'data': [{'st_histogram': {'b1': [[0, 29656.164705882355],
     [1, 16358.619607843137],
     [2, 47181.54509803922],
     [3, 0],
     [4, 6177.6]]}}],
 'meta': {}}

In [56]:
display(Markdown('#### Function ST_valueCount'))
queryRaster('fc8cf356-f4be-4635-a896-fb468aaaa832', "SELECT ST_valueCount(rast, 1, true) FROM 'users/resourcewatch/for_013_forest_landscape_restoration_opportunity'", '605fcf0e44c02de39fcaa7e30884be07')

#### Function ST_valueCount

{'data': [{'st_valuecount': {'b1': {'0': 29658,
     '1': 16322,
     '2': 47187,
     '4': 6178}}}],
 'meta': {}}

### Image Collections

ST_METADATA, st_SUMMARYSTATS are not supported per se; you can obtain the same results using count and count, max, mean, min, sum, stdev functions. or accessing the fields endpoint

Supported functions based on band:  
'avg': ee.Reducer.mean,  
'mean': ee.Reducer.mean,  
'max': ee.Reducer.max,  
'min': ee.Reducer.min,  
'var': ee.Reducer.variance,  
'stdev': ee.Reducer.stdDev,  
'count': ee.Reducer.count,  
'sum': ee.Reducer.sum,  
'mode': ee.Reducer.mode,  
'st_histogram': ee.Reducer.autoHistogram,  
'st_valuecount': ee.Reducer.frequencyHistogram,  
'every': ee.Reducer.countEvery,  
'first': ee.Reducer.first,  
'last': ee.Reducer.last,  
'frequency': ee.Reducer.frequencyHistogram,  
'array_agg': ee.Reducer.toList  

Capabilities to filter, reduce and aggregate based on properties.

http://api.resourcewatch.org/v1/fields/997a440a-fdb8-48c4-9eea-60619c934914

In [70]:
display(Markdown('#### Function st_summarystats replacement:'))
queryRaster('997a440a-fdb8-48c4-9eea-60619c934914', "select avg(q25), min(q50), avg(q75), 'system:index', RCP from 'projects/resource-watch-gee/cli_051_nexgddp_annual_tasmin' where ST_INTERSECTS(ST_SetSRID(ST_GeomFromGeoJSON('{\"type\":\"Point\",\"coordinates\":[-110.220,19.986]}'),4326),the_geom) GROUP BY system:index")

#### Function st_summarystats replacement:

{'data': [{'RCP': '4.5',
   'q25_mean': 17.010589599609375,
   'q50_min': 17.079345703125,
   'q75_mean': 17.116851806640625},
  {'RCP': '4.5',
   'q25_mean': 17.131103515625,
   'q50_min': 17.2337646484375,
   'q75_mean': 17.407867431640625},
  {'RCP': '4.5',
   'q25_mean': 17.238983154296875,
   'q50_min': 17.408050537109375,
   'q75_mean': 17.558563232421875},
  {'RCP': '4.5',
   'q25_mean': 17.380615234375,
   'q50_min': 17.566375732421875,
   'q75_mean': 17.704620361328125},
  {'RCP': '4.5',
   'q25_mean': 17.625030517578125,
   'q50_min': 17.812835693359375,
   'q75_mean': 17.905487060546875},
  {'RCP': '4.5',
   'q25_mean': 17.845703125,
   'q50_min': 18.024505615234375,
   'q75_mean': 18.21923828125},
  {'RCP': '4.5',
   'q25_mean': 17.944183349609375,
   'q50_min': 18.10992431640625,
   'q75_mean': 18.425750732421875},
  {'RCP': '4.5',
   'q25_mean': 18.03387451171875,
   'q50_min': 18.143646240234375,
   'q75_mean': 18.621307373046875},
  {'RCP': '4.5',
   'q25_mean': 18.0120

In [67]:
display(Markdown('#### Function st_summarystats replacement:'))
queryRaster('997a440a-fdb8-48c4-9eea-60619c934914', "select avg(q25), min(q50), avg(q75) from 'projects/resource-watch-gee/cli_051_nexgddp_annual_tasmin'")

#### Function st_summarystats replacement:

{'data': [{'q25_mean_mean': 1.338038881996025,
   'q50_min_min': -38.278350830078125,
   'q75_mean_mean': 2.097723852852547}],
 'meta': {}}

In [56]:
display(Markdown('#### Function st_histogram replacement:'))
queryRaster('997a440a-fdb8-48c4-9eea-60619c934914', "select st_histogram(q25, auto), 'system:index', RCP from 'projects/resource-watch-gee/cli_051_nexgddp_annual_tasmin' GROUP BY system:index limit 1")

#### Function st_histogram replacement:

{'data': [{'RCP': '4.5',
   'q25': [[-38.5, 1237],
    [-38, 618],
    [-37.5, 619],
    [-37, 618],
    [-36.5, 310],
    [-36, 927],
    [-35.5, 619],
    [-35, 309],
    [-34.5, 1237],
    [-34, 1236],
    [-33.5, 1546],
    [-33, 1237],
    [-32.5, 619],
    [-32, 619],
    [-31.5, 928],
    [-31, 618],
    [-30.5, 1236],
    [-30, 619],
    [-29.5, 927],
    [-29, 928],
    [-28.5, 619],
    [-28, 618],
    [-27.5, 1237],
    [-27, 6184],
    [-26.5, 8349],
    [-26, 1856],
    [-25.5, 2782],
    [-25, 2784],
    [-24.5, 1855],
    [-24, 927],
    [-23.5, 619],
    [-23, 619],
    [-22.5, 618],
    [-22, 1236],
    [-21.5, 619],
    [-21, 310],
    [-20.5, 928],
    [-20, 1237],
    [-19.5, 618],
    [-19, 618],
    [-18.5, 309],
    [-18, 0],
    [-17.5, 927],
    [-17, 0],
    [-16.5, 929],
    [-16, 927],
    [-15.5, 1237],
    [-15, 618],
    [-14.5, 1545],
    [-14, 309],
    [-13.5, 1238],
    [-13, 1547],
    [-12.5, 618],
    [-12, 0],
    [-11.5, 309],
    [-11, 928],
   

In [55]:
display(Markdown('#### Function st_histogram replacement:'))

queryRaster('b75d8398-34f2-447d-832d-ea570451995a',
            "select st_histogram(tropospheric_NO2_column_number_density, auto) as t from 'projects/resource-watch-gee/cit_035_tropomi_atmospheric_chemistry_model_30day_avg/NO2' where system:time_start >= 1533448800000", 
            '605fcf0e44c02de39fcaa7e30884be07' )

#### Function st_histogram replacement:

500 Server Error: Internal Server Error for url: https://api.resourcewatch.org/v1/query/b75d8398-34f2-447d-832d-ea570451995a?sql=select+st_histogram%28tropospheric_NO2_column_number_density%2C+auto%29+as+t+from+%27projects%2Fresource-watch-gee%2Fcit_035_tropomi_atmospheric_chemistry_model_30day_avg%2FNO2%27+where+system%3Atime_start+%3E%3D+1533448800000&geostore=605fcf0e44c02de39fcaa7e30884be07
